In [1]:
# Libraries
import numpy as np
import pandas as pd
import re

# Primera visualización rápida

Cargamos datos de formato CSV

In [2]:
attacks = pd.read_csv("../data/attacks.csv", encoding = "ISO-8859-1")

Visualizo las primeras y últimas filas

In [3]:
attacks.head(3)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN


In [4]:
attacks.tail(3)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
25720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25722,xx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Visualizo muestra aleatoria

In [5]:
attacks.sample(3)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
23858,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10557,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5088,1929.01.05,05-Jan-1929,1929.0,Unprovoked,FIJI,Viti Levu Island,Suva Harbor,Diving for coins thrown from ship S.S. Moeraki,male,M,...,NaN,"V.M. Coppleson (1962), p.253, possibly GSAF 19...",1929.01.05-FijianBoy.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1929.01.05,1929.01.05,1215.0,NaN,NaN


Decide a HYPOTHESIS (or hypotheses) with which you will clean the data


# Mis HIPOTESIS.

- USA es donde más accidentes de tiburón se producen.
- La mayor parte de los accidentes son no provocados.
- Fishing y Surfing; mayores causas de accidentes.
- Mueren más hombres que mujeres.

# Análisis exploratorio DataFrame

In [6]:
attacks.shape

(25723, 24)

In [7]:
attacks.info

<bound method DataFrame.info of       Case Number         Date    Year        Type    Country  \
0      2018.06.25  25-Jun-2018  2018.0     Boating        USA   
1      2018.06.18  18-Jun-2018  2018.0  Unprovoked        USA   
2      2018.06.09  09-Jun-2018  2018.0     Invalid        USA   
3      2018.06.08  08-Jun-2018  2018.0  Unprovoked  AUSTRALIA   
4      2018.06.04  04-Jun-2018  2018.0    Provoked     MEXICO   
...           ...          ...     ...         ...        ...   
25718         NaN          NaN     NaN         NaN        NaN   
25719         NaN          NaN     NaN         NaN        NaN   
25720         NaN          NaN     NaN         NaN        NaN   
25721         NaN          NaN     NaN         NaN        NaN   
25722          xx          NaN     NaN         NaN        NaN   

                  Area                        Location     Activity  \
0           California     Oceanside, San Diego County     Paddling   
1              Georgia  St. Simon Island, Gly

In [8]:
attacks.isnull().sum()

Case Number               17021
Date                      19421
Year                      19423
Type                      19425
Country                   19471
Area                      19876
Location                  19961
Activity                  19965
Name                      19631
Sex                       19986
Age                       22252
Injury                    19449
Fatal (Y/N)               19960
Time                      22775
Species                   22259
Investigator or Source    19438
pdf                       19421
href formula              19422
href                      19421
Case Number.1             19421
Case Number.2             19421
original order            19414
Unnamed: 22               25722
Unnamed: 23               25721
dtype: int64

In [9]:
attacks.describe()

,Year,original order
count,6300.000000,6309.000000
mean,1927.272381,3155.999683
std,281.116308,1821.396206
min,0.000000,2.000000
25%,1942.000000,1579.000000
50%,1977.000000,3156.000000
75%,2005.000000,4733.000000
max,2018.000000,6310.000000


COMPRUEBA DATOS NUMËRICOS Y CATEGÓRICOS

In [11]:
# DATOS NUMÉRICOS
numericas = attacks._get_numeric_data()
numericas.head(3)

,Year,original order
0,2018.0,6303.0
1,2018.0,6302.0
2,2018.0,6301.0


In [12]:
# DATOS CATEGÓRICOS
attacks.select_dtypes(include="object").head()

,Case Number,Date,Type,Country,Area,Location,Activity,Name,Sex,Age,...,Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,...,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,NaN,NaN
1,2018.06.18,18-Jun-2018,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,...,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,NaN,NaN
2,2018.06.09,09-Jun-2018,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,...,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,NaN,NaN
3,2018.06.08,08-Jun-2018,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,...,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,NaN,NaN
4,2018.06.04,04-Jun-2018,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,...,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,NaN,NaN


Renombro COLUMNAS

In [13]:
attacks_copia = attacks.copy()

In [14]:
columnas = list(attacks.columns)
columnas

['Case Number',
 'Date',
 'Year',
 'Type',
 'Country',
 'Area',
 'Location',
 'Activity',
 'Name',
 'Sex ',
 'Age',
 'Injury',
 'Fatal (Y/N)',
 'Time',
 'Species ',
 'Investigator or Source',
 'pdf',
 'href formula',
 'href',
 'Case Number.1',
 'Case Number.2',
 'original order',
 'Unnamed: 22',
 'Unnamed: 23']

In [15]:
diccio_nuevas = {columna: columna.replace(" ", "_").lower() for columna in columnas}

In [16]:
diccio_nuevas

{'Case Number': 'case_number',
 'Date': 'date',
 'Year': 'year',
 'Type': 'type',
 'Country': 'country',
 'Area': 'area',
 'Location': 'location',
 'Activity': 'activity',
 'Name': 'name',
 'Sex ': 'sex_',
 'Age': 'age',
 'Injury': 'injury',
 'Fatal (Y/N)': 'fatal_(y/n)',
 'Time': 'time',
 'Species ': 'species_',
 'Investigator or Source': 'investigator_or_source',
 'pdf': 'pdf',
 'href formula': 'href_formula',
 'href': 'href',
 'Case Number.1': 'case_number.1',
 'Case Number.2': 'case_number.2',
 'original order': 'original_order',
 'Unnamed: 22': 'unnamed:_22',
 'Unnamed: 23': 'unnamed:_23'}

In [17]:
attacks_renombrado = attacks.rename(columns=diccio_nuevas)
attacks_renombrado.sample()

,case_number,date,year,type,country,area,location,activity,name,sex_,...,species_,investigator_or_source,pdf,href_formula,href,case_number.1,case_number.2,original_order,unnamed:_22,unnamed:_23
21852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
attacks.rename(columns=diccio_nuevas, inplace=True)

In [19]:
attacks.head(2)

,case_number,date,year,type,country,area,location,activity,name,sex_,...,species_,investigator_or_source,pdf,href_formula,href,case_number.1,case_number.2,original_order,unnamed:_22,unnamed:_23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN


In [20]:
attacks

,case_number,date,year,type,country,area,location,activity,name,sex_,...,species_,investigator_or_source,pdf,href_formula,href,case_number.1,case_number.2,original_order,unnamed:_22,unnamed:_23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Elimino FILAS y COLUMNAS

In [21]:
# Compruebo los datos que no son Nan en la columna unnamed:_23.
attacks[attacks["unnamed:_23"].notna()]

,case_number,date,year,type,country,area,location,activity,name,sex_,...,species_,investigator_or_source,pdf,href_formula,href,case_number.1,case_number.2,original_order,unnamed:_22,unnamed:_23
4415,1952.03.30,30-Mar-1952,1952.0,Unprovoked,NETHERLANDS ANTILLES,Curacao,NaN,Went to aid of child being menaced by the shark,A.J. Eggink,M,...,"Bull shark, 2.7 m [9'] was captured & dragged ...","J. Randall, p.352 in Sharks & Survival; H.D. B...",1952.03.30-Eggink.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1952.03.30,1952.03.30,1888.0,NaN,Teramo
5840,1878.09.14.R,Reported 14-Sep-1878,1878.0,Provoked,USA,Connecticut,"Branford, New Haven County",Fishing,Captain Pattison,M,...,NaN,"St. Joseph Herald, 9/14/1878",1878.09.14.R-Pattison.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1878.09.14.R,1878.09.14.R,463.0,NaN,change filename


ELIMINO columnas que no me aportan valor.

In [22]:
attacks.drop(["pdf", "href_formula","href","unnamed:_22", "unnamed:_23"],axis=1,inplace=True)

In [23]:
attacks.head()

,case_number,date,year,type,country,area,location,activity,name,sex_,age,injury,fatal_(y/n),time,species_,investigator_or_source,case_number.1,case_number.2,original_order
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25,2018.06.25,6303.0
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18,2018.06.18,6302.0
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09,2018.06.09,6301.0
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08,2018.06.08,6300.0
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04,2018.06.04,6299.0


Además, elimino las siguientes columnas que no aportan nada a mis hipótesis.

In [24]:
attacks.drop(["time", "species_","investigator_or_source","case_number.1", "case_number.2", "original_order"],axis=1,inplace=True)

In [25]:
attacks.head()

,case_number,date,year,type,country,area,location,activity,name,sex_,age,injury,fatal_(y/n)
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N


Elimino las filas que tienen todo NaN

In [26]:
sharks = attacks.dropna(how="all", axis=0)

In [27]:
sharks.shape

(8703, 13)

LIMPIEZA columna; case_number

In [28]:
case_number = list(attacks.case_number.unique())

In [29]:
def quitar_a(x):
    if ".a" in str(x):
        return x.replace(".a", "")
    else:
        return x

In [30]:
sharks["case_number"] = sharks.case_number.apply(quitar_a)

<ipython-input-30-b9e4096ae751>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["case_number"] = sharks.case_number.apply(quitar_a)


In [31]:
def quitar_b(x):
    if ".b" in str(x):
        return x.replace(".b", "")
    else:
        return x

In [32]:
sharks["case_number"] = sharks.case_number.apply(quitar_b)

<ipython-input-32-7a0d2429a33b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["case_number"] = sharks.case_number.apply(quitar_b)


In [33]:
def quitar_c(x):
    if ".c" in str(x):
        return x.replace(".c", "")
    else:
        return x

In [34]:
sharks["case_number"] = sharks.case_number.apply(quitar_c)

<ipython-input-34-560dba157aa4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["case_number"] = sharks.case_number.apply(quitar_c)


In [35]:
def quitar_d(x):
    if ".c" in str(x):
        return x.replace(".d", "")
    else:
        return x

In [36]:
sharks["case_number"] = sharks.case_number.apply(quitar_d)

<ipython-input-36-327631ea2d19>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["case_number"] = sharks.case_number.apply(quitar_d)


In [37]:
def quitar_R(x):
    if ".R" in str(x):
        return x.replace(".R", "")
    else:
        return x

In [38]:
sharks["case_number"] = sharks.case_number.apply(quitar_R)

<ipython-input-38-46223fb203ba>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["case_number"] = sharks.case_number.apply(quitar_R)


LIMPIEZA columna; date

In [39]:
sharks['newcol'] = sharks['date'].str.extract(r"(\w+-\d+)")

<ipython-input-39-65ad4d2e00fd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks['newcol'] = sharks['date'].str.extract(r"(\w+-\d+)")


In [40]:
sharks.sample(3)

,case_number,date,year,type,country,area,location,activity,name,sex_,age,injury,fatal_(y/n),newcol
1158,2009.02.08,08-Feb-2009,2009.0,Sea Disaster,USA,Puerto Rico,Quebradillas,Air Disaster,occupant of a Cessna 206,M,NaN,It is probable that all 5 passengers died on i...,Y,Feb-2009
3980,1960.07.03,03-Jul-1960,1960.0,Provoked,USA,Virginia,Hog Island,"Fishing, tossing netted shark onboard",Elsworth Smith,M,41,Right arm bitten PROVOKED INCIDENT,N,Jul-1960
1819,2002.10.05,05-Oct-2002,2002.0,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Ivan Rios,M,35,Heel & back of right knee lacerated,N,Oct-2002


In [41]:
dates = list(sharks.date)
print(dates[0:5])

['25-Jun-2018', '18-Jun-2018', '09-Jun-2018', '08-Jun-2018', '04-Jun-2018']


In [42]:
newcoll= list(sharks.newcol)
print(newcoll[0:5])

['Jun-2018', 'Jun-2018', 'Jun-2018', 'Jun-2018', 'Jun-2018']


In [43]:
sharks.date=newcoll

/home/jesus_chao/anaconda3/envs/ironhack/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [44]:
print(dates[0:5])

['25-Jun-2018', '18-Jun-2018', '09-Jun-2018', '08-Jun-2018', '04-Jun-2018']


In [45]:
sharks.sample()

,case_number,date,year,type,country,area,location,activity,name,sex_,age,injury,fatal_(y/n),newcol
2413,1994.11.25,Nov-1994,1994.0,Unprovoked,SOUTH AFRICA,Western Cape Province,Mossel Bay,Spearfishing,Anton Bosman,M,NaN,No injury,N,Nov-1994


In [46]:
sharks.drop(["newcol"],axis=1, inplace=True)

/home/jesus_chao/anaconda3/envs/ironhack/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [47]:
sharks.sample()

,case_number,date,year,type,country,area,location,activity,name,sex_,age,injury,fatal_(y/n)
7748,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


LIMPIEZA columna; year

In [48]:
def year_(x):
    if ".0" in str(x):
        return str(x).replace(".0", "")
    else:
        return x

In [49]:
sharks["year"] = sharks.year.apply(year_)

<ipython-input-49-34703d824513>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["year"] = sharks.year.apply(year_)


In [50]:
print(sharks.year)

0        2018
1        2018
2        2018
3        2018
4        2018
         ... 
8698      NaN
8699      NaN
8700      NaN
8701      NaN
25722     NaN
Name: year, Length: 8703, dtype: object


In [51]:
sharks.year.unique()

array(['2018', '2017', nan, '2016', '2015', '2014', '2013', '2012',
       '2011', '2010', '2009', '2008', '2007', '2006', '2005', '2004',
       '2003', '2002', '2001', '2000', '1999', '1998', '1997', '1996',
       '1995', '1984', '1994', '1993', '1992', '1991', '1990', '1989',
       '1969', '1988', '1987', '1986', '1985', '1983', '1982', '1981',
       '1980', '1979', '1978', '1977', '1976', '1975', '1974', '1973',
       '1972', '1971', '1970', '1968', '1967', '1966', '1965', '1964',
       '1963', '1962', '1961', '1960', '1959', '1958', '1957', '1956',
       '1955', '1954', '1953', '1952', '1951', '1950', '1949', '1948',
       '1848', '1947', '1946', '1945', '1944', '1943', '1942', '1941',
       '1940', '1939', '1938', '1937', '1936', '1935', '1934', '1933',
       '1932', '1931', '1930', '1929', '1928', '1927', '1926', '1925',
       '1924', '1923', '1922', '1921', '1920', '1919', '1918', '1917',
       '1916', '1915', '1914', '1913', '1912', '1911', '1910', '1909',
       '1

LIMPÎEZA Columna type.

In [54]:
sharks.type.unique()

array(['Boating', 'Unprovoked', 'Invalid', 'Provoked', 'Questionable',
       'Sea Disaster', nan, 'Boat', 'Boatomg'], dtype=object)

In [55]:
def type_(x):
    if "Unprovoked" in str(x):
        return x.replace("Unprovoked", "Accidental")
    else:
        return x

In [56]:
sharks["type"] = sharks.type.apply(type_)

<ipython-input-56-e5a48b295325>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["type"] = sharks.type.apply(type_)


In [57]:
multi_p = sharks[(sharks.type == "Boatomg")]

In [58]:
multi_p

,case_number,date,year,type,country,area,location,activity,name,sex_,age,injury,fatal_(y/n)
1152,2009.03.01,Mar-2009,2009,Boatomg,NEW ZEALAND,North Island,Taranaki,Fishing,"boat, occupants: Boyd Rutherford & Hamish Roper",M,NaN,"No injury to occupants, shark bit propeller",N


In [59]:
def type_(x):
    if "Boatomg" in str(x):
        return x.replace("Boatomg", "Boat")
    else:
        return x

In [60]:
sharks["type"] = sharks.type.apply(type_)

<ipython-input-60-e5a48b295325>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["type"] = sharks.type.apply(type_)


In [61]:
def type_(x):
    if "Invalid" in str(x):
        return x.replace("Invalid", "Injured/Fatal")
    else:
        return x

In [62]:
sharks["type"] = sharks.type.apply(type_)

<ipython-input-62-e5a48b295325>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["type"] = sharks.type.apply(type_)


In [63]:
sharks.type.unique()

array(['Boating', 'Accidental', 'Injured/Fatal', 'Provoked',
       'Questionable', 'Sea Disaster', nan, 'Boat'], dtype=object)

In [86]:
sharks.type.value_counts()

Accidental       4595
Provoked          574
Injured/Fatal     547
Sea Disaster      239
Boating           203
Boat              138
Questionable        2
Name: type, dtype: int64

LIMPIEZA Pais, área y localización

In [64]:
countries = list(sharks.country.unique())

In [65]:
len(countries)

213

In [66]:
print(countries)

['USA', 'AUSTRALIA', 'MEXICO', 'BRAZIL', 'ENGLAND', 'SOUTH AFRICA', 'THAILAND', 'COSTA RICA', 'MALDIVES', 'BAHAMAS', 'NEW CALEDONIA', 'ECUADOR', 'MALAYSIA', 'LIBYA', nan, 'CUBA', 'MAURITIUS', 'NEW ZEALAND', 'SPAIN', 'SAMOA', 'SOLOMON ISLANDS', 'JAPAN', 'EGYPT', 'ST HELENA, British overseas territory', 'COMOROS', 'REUNION', 'FRENCH POLYNESIA', 'UNITED KINGDOM', 'UNITED ARAB EMIRATES', 'PHILIPPINES', 'INDONESIA', 'CHINA', 'COLUMBIA', 'CAPE VERDE', 'Fiji', 'DOMINICAN REPUBLIC', 'CAYMAN ISLANDS', 'ARUBA', 'MOZAMBIQUE', 'FIJI', 'PUERTO RICO', 'ITALY', 'ATLANTIC OCEAN', 'GREECE', 'ST. MARTIN', 'FRANCE', 'PAPUA NEW GUINEA', 'TRINIDAD & TOBAGO', 'KIRIBATI', 'ISRAEL', 'DIEGO GARCIA', 'TAIWAN', 'JAMAICA', 'PALESTINIAN TERRITORIES', 'GUAM', 'SEYCHELLES', 'BELIZE', 'NIGERIA', 'TONGA', 'SCOTLAND', 'CANADA', 'CROATIA', 'SAUDI ARABIA', 'CHILE', 'ANTIGUA', 'KENYA', 'RUSSIA', 'TURKS & CAICOS', 'UNITED ARAB EMIRATES (UAE)', 'AZORES', 'SOUTH KOREA', 'MALTA', 'VIETNAM', 'MADAGASCAR', 'PANAMA', 'SOMALIA', 

Ponemos todos los países en mayúsculas.

In [67]:
def upper_c(x):
    return str(x).upper()

In [68]:
sharks["country"] = sharks.country.apply(upper_c)

<ipython-input-68-51f75b38b595>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["country"] = sharks.country.apply(upper_c)


Reemplazar ENGLAND por UNITED KINGDOM

In [69]:
def england_(x):
    if "ENGLAND" in str(x):
        return x.replace("ENGLAND", "UNITED KINGDOM")
    else:
        return x

In [70]:
sharks["country"] = sharks.country.apply(england_)

<ipython-input-70-9a5641fac9da>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["country"] = sharks.country.apply(england_)


In [85]:
sharks.country.value_counts()

NAN                      2451
USA                      2229
AUSTRALIA                1338
SOUTH AFRICA              579
PAPUA NEW GUINEA          134
                         ... 
DIEGO GARCIA                1
ARGENTINA                   1
THE BALKANS                 1
NORTH ATLANTIC OCEAN        1
CEYLON (SRI LANKA)          1
Name: country, Length: 206, dtype: int64

ELIMINO signo interrogación

In [71]:
def question_(x):
    if "?" in str(x):
        return x.replace("?", "")
    else:
        return x

In [73]:
sharks["country"] = sharks.country.apply(question_)

<ipython-input-73-1d540e1a7bd3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["country"] = sharks.country.apply(question_)


In [74]:
sharks.country.unique()

array(['USA', 'AUSTRALIA', 'MEXICO', 'BRAZIL', 'UNITED KINGDOM',
       'SOUTH AFRICA', 'THAILAND', 'COSTA RICA', 'MALDIVES', 'BAHAMAS',
       'NEW CALEDONIA', 'ECUADOR', 'MALAYSIA', 'LIBYA', 'NAN', 'CUBA',
       'MAURITIUS', 'NEW ZEALAND', 'SPAIN', 'SAMOA', 'SOLOMON ISLANDS',
       'JAPAN', 'EGYPT', 'ST HELENA, BRITISH OVERSEAS TERRITORY',
       'COMOROS', 'REUNION', 'FRENCH POLYNESIA', 'UNITED ARAB EMIRATES',
       'PHILIPPINES', 'INDONESIA', 'CHINA', 'COLUMBIA', 'CAPE VERDE',
       'FIJI', 'DOMINICAN REPUBLIC', 'CAYMAN ISLANDS', 'ARUBA',
       'MOZAMBIQUE', 'PUERTO RICO', 'ITALY', 'ATLANTIC OCEAN', 'GREECE',
       'ST. MARTIN', 'FRANCE', 'PAPUA NEW GUINEA', 'TRINIDAD & TOBAGO',
       'KIRIBATI', 'ISRAEL', 'DIEGO GARCIA', 'TAIWAN', 'JAMAICA',
       'PALESTINIAN TERRITORIES', 'GUAM', 'SEYCHELLES', 'BELIZE',
       'NIGERIA', 'TONGA', 'SCOTLAND', 'CANADA', 'CROATIA',
       'SAUDI ARABIA', 'CHILE', 'ANTIGUA', 'KENYA', 'RUSSIA',
       'TURKS & CAICOS', 'UNITED ARAB EMIRATES (

In [75]:
activities = list(sharks.activity.unique())

In [76]:
activities

['Paddling',
 'Standing',
 'Surfing',
 'Free diving',
 'Kite surfing',
 'Swimming',
 'Fishing',
 'Walking',
 'Feeding sharks',
 'Boogie boarding',
 'Scuba diving',
 'Paddle-skiing',
 'Body boarding',
 'Windsurfing',
 'Stand-Up Paddleboarding',
 'Wading',
 'Scuba Diving',
 'Kayak fishing for sharks',
 'Cleaning fish',
 'Snorkeling',
 'Spearfishing',
 nan,
 'Diving',
 'Fishing / Wading',
 '2 boats capsized',
 'Night bathing',
 'Surfing ',
 'Kayaking / Fishing',
 'Kayaking',
 'Body surfing',
 'Swimming, poaching abalone',
 'Canoeing',
 'SUP',
 'Skimboarding',
 'Touching a shark',
 'Fishing ',
 'Attempting to lasso a shark',
 'Photo shoot',
 'Paddle boarding',
 'Kakaying',
 'Washing hands',
 'Grabbing shark for a selfie',
 'Kayak fishing',
 'Tagging sharks',
 'Surf skiing ',
 'Surf fishing',
 'Floating',
 'Cage Diving',
 'Surfng',
 'SUP Foil boarding',
 'Lobstering',
 'Fishing in Alabama Deep Fishing Rodeo',
 'Fishing for squid',
 'Fishing for sharks',
 'Floating in tube',
 'Teasing a shar

In [77]:
sharks["fishing"] = sharks["activity"].str.contains("ishing")

<ipython-input-77-bfcf38036315>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["fishing"] = sharks["activity"].str.contains("ishing")


In [78]:
sharks.head()

,case_number,date,year,type,country,area,location,activity,name,sex_,age,injury,fatal_(y/n),fishing
0,2018.06.25,Jun-2018,2018,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,False
1,2018.06.18,Jun-2018,2018,Accidental,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,False
2,2018.06.09,Jun-2018,2018,Injured/Fatal,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,False
3,2018.06.08,Jun-2018,2018,Accidental,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,False
4,2018.06.04,Jun-2018,2018,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,False


In [79]:
sharks["surfing"] = sharks["activity"].str.contains("urfing")

<ipython-input-79-d92449e05581>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["surfing"] = sharks["activity"].str.contains("urfing")


In [80]:
sharks.head()

,case_number,date,year,type,country,area,location,activity,name,sex_,age,injury,fatal_(y/n),fishing,surfing
0,2018.06.25,Jun-2018,2018,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,False,False
1,2018.06.18,Jun-2018,2018,Accidental,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,False,False
2,2018.06.09,Jun-2018,2018,Injured/Fatal,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,False,True
3,2018.06.08,Jun-2018,2018,Accidental,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,False,True
4,2018.06.04,Jun-2018,2018,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,False,False


In [87]:
sharks.fishing.value_counts()

False    4586
True     1172
Name: fishing, dtype: int64

In [88]:
sharks.surfing.value_counts()

False    4630
True     1128
Name: surfing, dtype: int64

In [90]:
sharks.isnull().sum()

case_number       1
date           2946
year           2403
type           2405
country           0
area           2856
location       2941
activity       2945
name           2611
sex_           2966
age            5232
injury         2429
fatal_(y/n)    2940
fishing        2945
surfing        2945
dtype: int64

2945 nulls

In [91]:
suma_fish_surf = 1128 + 1172

In [94]:
Porcentaje = suma_fish_surf/(4630+1128)

In [95]:
Porcentaje

0.399444251476207

Limpieza de la columna sex_

In [152]:
gender = list(sharks.sex_.unique())

In [153]:
gender

['F', 'M', nan, 'M ', 'lli', 'N', '.']

In [141]:
sharks.drop(["newcol"],axis=1,inplace=True)

/home/jesus_chao/anaconda3/envs/ironhack/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [142]:
sharks.head()

,case_number,date,year,type,country,area,location,activity,name,sex_,age,injury,fatal_(y/n),fishing,surfing
0,2018.06.25,Jun-2018,2018,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,False,False
1,2018.06.18,Jun-2018,2018,Accidental,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,False,False
2,2018.06.09,Jun-2018,2018,Injured/Fatal,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,False,True
3,2018.06.08,Jun-2018,2018,Accidental,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,False,True
4,2018.06.04,Jun-2018,2018,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,False,False


In [147]:
def gender_(x):
    special_characters = 'FM'
    if str(x) in special_characters:
        return x
    else:
        return str(x).replace(str(x), "None")

In [154]:
sharks.drop(["newcol"],axis=1,inplace=True)

/home/jesus_chao/anaconda3/envs/ironhack/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [156]:
sharks["genero"] = sharks.sex_.apply(gender_)

<ipython-input-156-0220de8c0124>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["genero"] = sharks.sex_.apply(gender_)


In [157]:
sharks.head()

,case_number,date,year,type,country,area,location,activity,name,sex_,age,injury,fatal_(y/n),fishing,surfing,genero
0,2018.06.25,Jun-2018,2018,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,False,False,F
1,2018.06.18,Jun-2018,2018,Accidental,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,False,False,F
2,2018.06.09,Jun-2018,2018,Injured/Fatal,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,False,True,M
3,2018.06.08,Jun-2018,2018,Accidental,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,False,True,M
4,2018.06.04,Jun-2018,2018,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,False,False,M


In [159]:
gender = list(sharks.genero.unique())

In [160]:
gender

['F', 'M', 'None']

In [161]:
sharks.drop(["sex_"],axis=1,inplace=True)

/home/jesus_chao/anaconda3/envs/ironhack/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [162]:
sharks.head()

,case_number,date,year,type,country,area,location,activity,name,age,injury,fatal_(y/n),fishing,surfing,genero
0,2018.06.25,Jun-2018,2018,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,57,"No injury to occupant, outrigger canoe and pad...",N,False,False,F
1,2018.06.18,Jun-2018,2018,Accidental,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,11,Minor injury to left thigh,N,False,False,F
2,2018.06.09,Jun-2018,2018,Injured/Fatal,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,48,Injury to left lower leg from surfboard skeg,N,False,True,M
3,2018.06.08,Jun-2018,2018,Accidental,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,NaN,Minor injury to lower leg,N,False,True,M
4,2018.06.04,Jun-2018,2018,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,False,False,M


In [163]:
sharks.genero.value_counts()

M       5094
None    2972
F        637
Name: genero, dtype: int64

OPERA CON COLUMNAS (SELECCIONA, ELIMINA, Valores únicos ...)

Filtramos los datos por el valor de las columnas

In [ ]:
attacks.area.unique()

BORRAR FILAS Y COLUMNAS. Reset_index y set_index

APPLY!!

Df .loc e .iloc
Seleccionar filas y columnas en Pandas mediante posición con iloc
Seleccionar filas y columnas en Pandas en base a etiquetas con loc

Ordenamos un dataframe
Buscamos valores nulos
Borramos los valores nulos
rellenar los datos que faltan
 Borramos los duplicados

 Agregación de datos
 GroupBy

EXPLORE the DATA and write down what you have found
you can use: df.describe(), df["column"], etc.

DRAW at least 2 GRAPHS that are insightful.

Use at least 5 data cleaning techniques inside a file named clean.ipynb
null values, columns drop, duplicated data, string manipulation, apply fn, categorize, regex, etc.

Show data that validates the conclusions based on your hypoteses in a file named analysis.ipynb

Build a compelling story-telling around your findings. Think of your stakeholders and convince them with your conclusions! (Some slides with few text and pretty plots are normally useful)